In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.jars", "/opt/spark/postgresql-42.7.8.jar") \
    .getOrCreate()

25/12/29 08:39:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/29 08:39:14 WARN DependencyUtils: Local jar /opt/spark/postgresql-42.7.8.jar does not exist, skipping.
25/12/29 08:39:15 INFO SparkContext: Running Spark version 3.4.0
25/12/29 08:39:15 INFO ResourceUtils: ==============================================================
25/12/29 08:39:15 INFO ResourceUtils: No custom resources configured for spark.driver.
25/12/29 08:39:15 INFO ResourceUtils: ==============================================================
25/12/29 08:39:15 INFO SparkContext: Submitted application: pyspark-shell
25/12/29 08:39:15 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
2

In [2]:
schema_ddl="run INT, event DOUBLE, E1 DOUBLE, px1 DOUBLE, py1 double, pz1 double, pt1 DOUBLE, eta1 DOUBLE, phi1 DOUBLE, Q1 INT, E2 DOUBLE ,px2 DOUBLE, py2 DOUBLE, pz2 DOUBLE ,pt2 DOUBLE, eta2 DOUBLE ,phi2 DOUBLE ,Q2 INT, M DOUBLE"
df = spark.read.csv("dielectron.csv", header=True, schema=schema_ddl)

25/12/29 08:39:24 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/12/29 08:39:24 INFO SharedState: Warehouse path is 'file:/opt/spark/work-dir/spark-warehouse'.
25/12/29 08:39:25 INFO InMemoryFileIndex: It took 87 ms to list leaf files for 1 paths.


In [3]:
df.head(1)

25/12/29 08:39:35 INFO FileSourceStrategy: Pushed Filters: 
25/12/29 08:39:35 INFO FileSourceStrategy: Post-Scan Filters: 
25/12/29 08:39:36 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 199.8 KiB, free 434.2 MiB)
25/12/29 08:39:36 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 434.2 MiB)
25/12/29 08:39:36 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 5d662929a363:44453 (size: 34.4 KiB, free: 434.4 MiB)
25/12/29 08:39:36 INFO SparkContext: Created broadcast 0 from head at /tmp/ipykernel_56/2384563279.py:1
25/12/29 08:39:36 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
25/12/29 08:39:36 INFO SparkContext: Starting job: head at /tmp/ipykernel_56/2384563279.py:1
25/12/29 08:39:36 INFO DAGScheduler: Got job 0 (head at /tmp/ipykernel_56/2384563279.py:1) with 1 output partitions
25/12/29 08:39:36 INFO DAGSc

[Row(run=147115, event=626787667.0, E1=2.55245, px1=0.783675, py1=-2.42902, pz1=-0.026877, pt1=2.55231, eta1=-0.0105303, phi1=-1.25871, Q1=1, E2=14.2767, px2=4.38735, py2=-13.5851, pz2=-0.142737, pt2=14.276, eta2=-0.0099982, phi2=-1.25842, Q2=-1, M=None)]

In [5]:
from pyspark.sql.functions import col
df = df.filter(col('M').isNotNull())

In [6]:
df.columns

['run',
 'event',
 'E1',
 'px1',
 'py1',
 'pz1',
 'pt1',
 'eta1',
 'phi1',
 'Q1',
 'E2',
 'px2',
 'py2',
 'pz2',
 'pt2',
 'eta2',
 'phi2',
 'Q2',
 'M']

In [7]:
df.printSchema()

root
 |-- run: integer (nullable = true)
 |-- event: double (nullable = true)
 |-- E1: double (nullable = true)
 |-- px1: double (nullable = true)
 |-- py1: double (nullable = true)
 |-- pz1: double (nullable = true)
 |-- pt1: double (nullable = true)
 |-- eta1: double (nullable = true)
 |-- phi1: double (nullable = true)
 |-- Q1: integer (nullable = true)
 |-- E2: double (nullable = true)
 |-- px2: double (nullable = true)
 |-- py2: double (nullable = true)
 |-- pz2: double (nullable = true)
 |-- pt2: double (nullable = true)
 |-- eta2: double (nullable = true)
 |-- phi2: double (nullable = true)
 |-- Q2: integer (nullable = true)
 |-- M: double (nullable = true)



In [8]:
df.write.jdbc(
    url="jdbc:postgresql://host.docker.internal:5432/postgres",
    table="raw_data",
    mode="overwrite",
    properties={
        "user": "postgres",
        "password": "password",
        "driver": "org.postgresql.Driver"
    }
)

25/12/29 08:49:19 INFO FileSourceStrategy: Pushed Filters: IsNotNull(M)
25/12/29 08:49:19 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(M#18)
25/12/29 08:49:19 INFO CodeGenerator: Code generated in 48.314266 ms
25/12/29 08:49:19 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 199.8 KiB, free 434.0 MiB)
25/12/29 08:49:19 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 433.9 MiB)
25/12/29 08:49:19 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 5d662929a363:44453 (size: 34.4 KiB, free: 434.3 MiB)
25/12/29 08:49:19 INFO SparkContext: Created broadcast 2 from jdbc at <unknown>:0
25/12/29 08:49:19 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
25/12/29 08:49:19 INFO SparkContext: Starting job: jdbc at <unknown>:0
25/12/29 08:49:19 INFO DAGScheduler: Got job 1 (jdbc at <unknown>:0) with 4 output partition